# Customization

In [43]:
from tensorflow import keras
import tensorflow as tf
from tensorflow.keras.initializers import Ones, Zeros

## Custom Layers

### Batch normalization

In [112]:
class BatchNormLayer(keras.layers.Layer):
    def __init__(self, axis=-1, exp_weight_factor=0.99, **kwargs):
        super().__init__(**kwargs)
        self.axis = axis
        self.exp_weight_factor = exp_weight_factor

    def reshape_weights(self, weights):
        return [tf.reshape(w, shape=self.weights_shape) for w in weights]

    def build(self, input_shape):
        self.weights_shape = [1]*len(input_shape)
        self.w_dim = input_shape[self.axis]
        self.weights_shape[self.axis] = self.w_dim
        self.axis = self.axis if self.axis >= 0 else len(input_shape)-1
        self.reduce_axes = [i for i in range(len(input_shape)) if i != self.axis]
        self.alpha = self.add_weight(shape=(self.w_dim,), initializer=Ones(), trainable=True)
        self.beta = self.add_weight(shape=(self.w_dim,), initializer=Zeros(), trainable=True)
        self.sigma = self.add_weight(shape=(self.w_dim,), initializer=Ones(), trainable=False)
        self.mu = self.add_weight(shape=(self.w_dim,), initializer=Zeros(), trainable=False)

    def _update_params(self, inputs):
        w = self.exp_weight_factor
        mu_batch = tf.math.reduce_mean(inputs, axis=self.reduce_axes)
        sigma_batch = tf.math.reduce_std(inputs, axis=self.reduce_axes)
        self.mu.assign(w*self.mu + (1-w)*mu_batch)
        self.sigma.assign(w*self.sigma + (1-w)*sigma_batch)
    
    def call(self, inputs, training=None):
        alpha, beta, sigma, mu = self.reshape_weights([self.alpha, self.beta, self.sigma, self.mu])
        if training:
            self._update_params(inputs)
        norm = (inputs-mu) / sigma
        res = (norm*alpha) + beta
        return res

    def get_config(self):
        return {'axis': self.axis, 'exp_weight_factor': exp_weight_factor}

In [113]:
bn = BatchNormLayer()

In [114]:
import numpy as np
x = np.random.rand(32, 10).astype(np.float32)*10 +7
out = bn(x)
out.shape

TensorShape([32, 10])

In [135]:
for i in range(1000):
    out = bn(x)

In [136]:
[np.std(out[i]) for i in range(out.shape[0])]

[0.85505694,
 0.84585935,
 0.76592386,
 0.81223124,
 0.62168145,
 0.8850148,
 0.8502795,
 1.1466483,
 0.6224386,
 1.1259699,
 1.0906398,
 0.887234,
 1.073935,
 1.0827506,
 0.74335426,
 0.86408806,
 0.8345361,
 1.0506405,
 0.5948313,
 1.2931309,
 0.9339112,
 1.0924976,
 0.9098994,
 1.2603775,
 0.830108,
 0.69927573,
 1.2257032,
 0.9891211,
 0.73111165,
 0.85022646,
 1.3250093,
 0.94862163]

In [ ]:
class DroputLayer(keras.layers.Layer):
    def __init__(self, keep_prob, **kwargs):
        super().__init__(**kwargs)
        self.keep_prob = keep_prob
        
    def call(self, inputs, training=None):


    def get_config(self):
        return {'keep_prob': self.keep_prob}